In [37]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
import numpy as np
import  pandas as pd

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,log_loss
from sklearn.model_selection import train_test_split
from pandas_ml import ConfusionMatrix as cm
from sklearn import svm
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')
import scipy.stats as st
import statsmodels as sm
import matplotlib

from IPython.display import display,clear_output,HTML

In [38]:
z=pd.read_csv('Feature_Vector_50_50_12_24_random_state_0.csv',index_col='User')


print("The feature vector for infected users and sample of uninfected users")
print('Infected/Unifected Users')
display(z.Target.value_counts())

for col in z.columns[:-1]:
    z[col]=z[col].apply(lambda x: np.log(x))
    
z.replace([np.inf,-np.inf],0,inplace=True)

The feature vector for infected users and sample of uninfected users
Infected/Unifected Users


1    2910
0    2910
Name: Target, dtype: int64

In [39]:
X=z.iloc[:,z.columns!='Target']
display(X.head(2))

### Build our target variable
print("1 -> Infected\n0 -> Uninfected")
Y=z.Target
# # Y=Y.apply(lambda x: 1 if x=='infected' else 0)
display(Y.head(5))

### Scale feature set  using Standard Scaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)


print("The train/test split ratio is 80:20")
X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,Y,random_state=0,test_size=0.2)


,SP,DP,NAPP,SIP,DIP,NREC,PAR,PAS,LEN,BYS,...,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300
User,,,,,,,,,,,,,,,,,,,,,
acbc3e760a890f55ceafd2573cde6f7c040dae93259a0cdfc7d1338dd60c6953,8.832734,4.234107,4.248495,4.624973,7.315218,8.921858,5.204284,4.250029,4.695142,9.834805,...,5.590987,5.988961,6.715383,5.733341,5.407172,5.575949,5.433722,4.828314,4.836282,5.402677
3c45372db59b842faec38488806b64c282270417f6755ba493e51eda8e98b402,2.564949,1.386294,1.386294,1.098612,2.079442,2.564949,4.582610,4.499810,4.482568,8.956371,...,0.000000,0.000000,1.945910,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


1 -> Infected
0 -> Uninfected


User
acbc3e760a890f55ceafd2573cde6f7c040dae93259a0cdfc7d1338dd60c6953    1
3c45372db59b842faec38488806b64c282270417f6755ba493e51eda8e98b402    0
653fe3e24024826e008a512560be7abb15b06bd46e89ba4159dbdc6b8b5983c3    0
8c8d63c29001941282625da51ded54c9305c2b2c5ed680ef803b61b1231a2107    1
ac87be58a25e50356ef8d66135f93392ac6c40f3c7b557e7c16f663ce168e9fc    1
Name: Target, dtype: int64

The train/test split ratio is 80:20


### Model on all of the features for all the users. (Just for an overview of the performace of the alogorithm)

In [40]:
clf = GaussianNB()
clf.fit(X_train, Y_train)
print("Accuracy on training data is: ",clf.score(X_train,Y_train))
print("Confusion matrix for Training data\n")
display(cm(Y_train.values,clf.predict(X_train)))

y_pred=clf.predict(X_test)
print("Accuracy on test data is: ",accuracy_score(Y_test,y_pred))
print("Confusion matrix for Test data\n")
display(cm(Y_test.values,y_pred))

print('Mean:',clf.theta_)
print("Variance:",clf.sigma_)

Accuracy on training data is:  0.79295532646
Confusion matrix for Training data



Predicted  False  True  __all__
Actual                         
False       1703   614     2317
True         350  1989     2339
__all__     2053  2603     4656

Accuracy on test data is:  0.806701030928
Confusion matrix for Test data



Predicted  False  True  __all__
Actual                         
False        452   141      593
True          84   487      571
__all__      536   628     1164

Mean: [[-0.58492875 -0.49711945 -0.52490072 -0.46793334 -0.57894502 -0.58740789
  -0.3668267  -0.31922742 -0.11182702 -0.29174031 -0.36105975 -0.62129367
  -0.60824901 -0.6017791  -0.59170074 -0.58287997 -0.57292219 -0.55562596
  -0.51790495 -0.49567025 -0.49757944 -0.50911801 -0.51901391 -0.53087017
  -0.5403378  -0.55549575 -0.56685017 -0.58221189 -0.59306359 -0.60546586
  -0.60959935 -0.62269318 -0.62406245 -0.62846212 -0.61999081]
 [ 0.5961036   0.50736617  0.53836554  0.48453579  0.59028365  0.59848727
   0.37727748  0.32232581  0.11600528  0.29744725  0.38265477  0.6215137
   0.60748444  0.61037077  0.60135662  0.59243695  0.58446354  0.56795004
   0.52832926  0.50185958  0.50910987  0.52248044  0.53591252  0.53883313
   0.55229283  0.5619444   0.57712936  0.59093936  0.60730355  0.62244359
   0.62444565  0.63187356  0.63036824  0.62863788  0.62302115]]
Variance: [[ 1.07285444  0.85474196  1.23421261  1.16478478  1.07727856  1.05252755
   1.20170109  1.22773589  1.33744419  1.243

### Malicious Users

In [41]:
z_mal=z[z.Target.values==1]

Y=z_mal.Target

for col in z_mal.columns[:-1]:
    X=z_mal[col].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X.reshape(-1,1))
    X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,Y,random_state=0,test_size=0.2)
    
    print("\nFeature -",col)
    clf = GaussianNB()
    clf.fit(X_train, Y_train)
#     print("Accuracy on training data is: ",clf.score(X_train,Y_train))
#     y_pred=clf.predict(X_test)
#     print("Accuracy on test data is: ",accuracy_score(Y_test,y_pred))
    print('Mean:',clf.theta_,"Variance:",clf.sigma_)


Feature - SP
Mean: [[-0.00189899]] Variance: [[ 1.0016999]]

Feature - DP
Mean: [[ 0.00233566]] Variance: [[ 1.01234126]]

Feature - NAPP
Mean: [[ 0.00032145]] Variance: [[ 1.00480726]]

Feature - SIP
Mean: [[ 0.00046486]] Variance: [[ 1.01381516]]

Feature - DIP
Mean: [[-0.00172513]] Variance: [[ 1.00527924]]

Feature - NREC
Mean: [[-0.00266679]] Variance: [[ 1.00161331]]

Feature - PAR
Mean: [[-0.00885456]] Variance: [[ 0.9704972]]

Feature - PAS
Mean: [[-0.00976446]] Variance: [[ 0.95693817]]

Feature - LEN
Mean: [[ 0.0034284]] Variance: [[ 1.01698901]]

Feature - BYS
Mean: [[-0.00452922]] Variance: [[ 0.97201583]]

Feature - BYR
Mean: [[-0.00763639]] Variance: [[ 0.9692453]]

Feature - 0000
Mean: [[-0.00605274]] Variance: [[ 1.00210774]]

Feature - 0100
Mean: [[-0.00369063]] Variance: [[ 1.0041438]]

Feature - 0200
Mean: [[-0.00733211]] Variance: [[ 1.01085482]]

Feature - 0300
Mean: [[-0.00236563]] Variance: [[ 1.00544312]]

Feature - 0400
Mean: [[-0.00215751]] Variance: [[ 1.002

### For non-malicious users

In [42]:
z_non=z[z.Target.values==0]

Y=z_non.Target

for col in z_non.columns[:-1]:
    X=z_non[col].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X.reshape(-1,1))
    X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,Y,random_state=0,test_size=0.2)
    
    print("\nFeature -",col)
    clf = GaussianNB()
    clf.fit(X_train, Y_train)
#     print("Accuracy on training data is: ",clf.score(X_train,Y_train))
#     y_pred=clf.predict(X_test)
#     print("Accuracy on test data is: ",accuracy_score(Y_test,y_pred))
    print('Mean:',clf.theta_,"Variance:",clf.sigma_)


Feature - SP
Mean: [[ 0.00116967]] Variance: [[ 0.99953347]]

Feature - DP
Mean: [[ 0.00417824]] Variance: [[ 0.99955403]]

Feature - NAPP
Mean: [[ 0.00597993]] Variance: [[ 0.98300427]]

Feature - SIP
Mean: [[-0.00376042]] Variance: [[ 1.00535477]]

Feature - DIP
Mean: [[ 0.00252891]] Variance: [[ 0.99521879]]

Feature - NREC
Mean: [[ 0.00163097]] Variance: [[ 0.99889635]]

Feature - PAR
Mean: [[-0.00185018]] Variance: [[ 0.98717047]]

Feature - PAS
Mean: [[ 0.00059443]] Variance: [[ 0.98819885]]

Feature - LEN
Mean: [[ 0.00842023]] Variance: [[ 0.98317932]]

Feature - BYS
Mean: [[  3.24580477e-05]] Variance: [[ 0.96494329]]

Feature - BYR
Mean: [[ 0.00337324]] Variance: [[ 0.9582504]]

Feature - 0000
Mean: [[-0.00284341]] Variance: [[ 0.98383548]]

Feature - 0100
Mean: [[-0.00472199]] Variance: [[ 0.98891223]]

Feature - 0200
Mean: [[-0.00533692]] Variance: [[ 0.99327216]]

Feature - 0300
Mean: [[-0.00024187]] Variance: [[ 0.995565]]

Feature - 0400
Mean: [[-0.00057567]] Variance: [

In [43]:
# ### For all Users on each feature individually

# ### One class classification with SVM - https://scikit-learn.org/stable/modules/generated/sklearn.svm.OneClassSVM.html#sklearn.svm.OneClassSVM

# ### For all malicious users

# z_mal=z[z.Target.values==1]

# Y=z_mal.Target

# for col in z_mal.columns[:-1]:
#     X=z_mal[col].values
#     scaler=StandardScaler()
#     scaled_X=scaler.fit_transform(X.reshape(-1,1))
#     X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,Y,random_state=0,test_size=0.2)
#     print("\nFeature -",col)
    
#     clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
#     clf.fit(X_train)
#     y_pred_train = clf.predict(X_train)
#     y_pred_test = clf.predict(X_test)
#     print("Accuracy on training data is: ",accuracy_score(Y_train,y_pred_train))
    
#     y_pred=clf.predict(X_test)
#     print("Accuracy on test data is: ",accuracy_score(Y_test,y_pred_test))

# ### For Non-Malicious Users

# z_non=z[z.Target.values==0]

# Y=z_non.Target.apply(lambda x: 1 if x==0 else 0)

# for col in z_non.columns[:-1]:
#     X=z_non[col].values
#     scaler=StandardScaler()
#     scaled_X=scaler.fit_transform(X.reshape(-1,1))
#     X_train, X_test, Y_train, Y_test = train_test_split(scaled_X,Y,random_state=0,test_size=0.2)
#     print("\nFeature -",col)
    
#     clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
#     clf.fit(X_train)
#     y_pred_train = clf.predict(X_train)
#     y_pred_test = clf.predict(X_test)
#     print("Accuracy on training data is: ",accuracy_score(Y_train,y_pred_train))
    
#     y_pred=clf.predict(X_test)
#     print("Accuracy on test data is: ",accuracy_score(Y_test,y_pred_test))

In [44]:
### Original Creator : Darshan Bhansali
### HTML code to hide the input cells 
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To toggle code, click <a href="javascript:code_toggle()">here</a>.''')